In [1]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from matplotlib.image import imread
import pandas as pd
from PIL import Image
import cv2

In [2]:
import zipfile
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
import warnings

2024-11-16 21:18:36.199199: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 21:18:36.203545: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 21:18:36.216509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731791916.235059    3382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731791916.240662    3382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 21:18:36.261670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
zip_path = 'FracAtlas.zip'
extract_dir = 'fracatlas_extracted'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

images_dir = os.path.join(extract_dir, 'FracAtlas/images')
annotations_dir = os.path.join(extract_dir, 'FracAtlas/Annotations/YOLO')

In [4]:
# Checked to see if each image has a corresponding label

categories = ['Fractured', 'Non_fractured']

for cat in categories:
    category_path = os.path.join(images_dir, cat)
    
    for img_file in os.listdir(category_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_name = os.path.splitext(img_file)[0]
            annotation_file = img_name + '.txt'
            annotation_path = os.path.join(annotations_dir, annotation_file)

            # if os.path.exists(annotation_path):
            #     print(f"Found annotation for {img_file}: {annotation_path}")
            # else:
            #     print(f"No annotation found for {img_file} at {annotation_path}.")

In [5]:
# Organize the data into {"inputs": images, "labels": labels} structure!

images_dir = 'fracatlas_extracted/FracAtlas/images'
annotations_dir = 'fracatlas_extracted/FracAtlas/Annotations/YOLO'

images = []
labels = []

for category in os.listdir(images_dir):
    category_path = os.path.join(images_dir, category)
    
    if not os.path.isdir(category_path):
        continue 

    for img_file in os.listdir(category_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')): 
            img_name = os.path.splitext(img_file)[0]
            annotation_file = img_name + '.txt'
            annotation_path = os.path.join(annotations_dir, annotation_file)

            if os.path.exists(annotation_path):
                img_path = os.path.join(category_path, img_file)
                images.append(img_path)
                labels.append(category)
            else:
                print(f"No annotation found for {img_file} at {annotation_path}.")

data_dict_old = {
    "inputs": images,
    "labels": labels
}

print(f"{len(images)} image paths and {len(labels)} labels in organized data")

4083 image paths and 4083 labels in organized data


In [6]:
# Do not include corrupted/damaged images

def preprocess(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = img_array / 255.0 # normalize pixel values
        return img_array
    except (OSError, IOError) as e:
        warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
        return None

images = []
labels = []

for img_path, label in zip(data_dict_old["inputs"], data_dict_old["labels"]):
    img = preprocess(img_path)
    if img is not None:
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

data_dict = {
    "inputs": images,
    "labels": labels
}

print(f"{len(images)}/{len(data_dict_old['inputs'])} images processed")

/tmp/ipykernel_3382/2169368734.py:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004189.jpg. Error: image file is truncated (16 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
/tmp/ipykernel_3382/2169368734.py:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004123.jpg. Error: image file is truncated (17 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
/tmp/ipykernel_3382/2169368734.py:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004297.jpg. Error: image file is truncated (5 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
/tmp/ipykernel_3382/2169368734.py:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004290.jpg. Error: image file is truncated (54 bytes not proce

4024/4083 images processed


In [7]:
# Encode the output labels to be 0 or 1

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data_dict["labels"])
data_dict["encoded_labels"] = encoded_labels

In [8]:
#image.load_img("fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004189.jpg", target_size=(224, 224))

In [9]:
#fracatlas_extracted/FracAtlas/Annotations/YOLO/IMG0000008.txt
#fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0000008.jpg